<a href="https://colab.research.google.com/github/shirinlatifi/pe-project/blob/master/RS_MLP%26GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
from deap import base, creator, tools, algorithms
import pandas as pd

In [3]:
df = pd.read_excel('solution gas.xlsx')

In [4]:
df['Pressure'] = df['Pressure'].fillna(df['Pressure'].mean())
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pressure  2160 non-null   float64
 1   Rs        2160 non-null   float64
dtypes: float64(2)
memory usage: 33.9 KB


In [5]:
df.corr()

,Pressure,Rs
Pressure,1.000000,0.845097
Rs,0.845097,1.000000


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler

In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('Rs', axis=1), df['Rs'], test_size=0.2, random_state=42)

In [11]:
# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = scaler.transform(np.array(y_train).reshape(-1,1))
y_test = scaler.transform(np.array(y_test).reshape(-1,1))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [12]:
#Define the MLP Architecture
def build_mlp(hidden_units):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train.shape[1],)))
    for units in hidden_units:
        model.add(tf.keras.layers.Dense(units, activation='relu'))
    model.add(tf.keras.layers.Dense(1))  # Output layer for regression
    return model

In [ ]:
build_mlp(3)

In [14]:
#Define the Fitness Function
def evaluate(individual):
    hidden_units = individual[0]  # Chromosome represents the hidden layer units
    mlp_model = build_mlp(hidden_units)
    mlp_model.compile(optimizer='adam', loss='mean_squared_error')
    mlp_model.fit(X_train, y_train, epochs=10, verbose=2)
    y_pred = mlp_model.predict(X_test)
    mse = np.mean(np.square(y_pred - y_test))
    return mse,

In [ ]:
evaluate()

In [ ]:
#prediction

In [ ]:
# acc and MSE

In [ ]:
#Visualization
import matplotlib.pyplot as plt

# Plotting the actual test data and predictions
plt.figure(figsize=(6,3))
plt.scatter(y_test, y_pred, color='blue', label='Predictions')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', linewidth=2, label='Ideal')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.legend()
plt.grid(True)
# plt. ('mse or line function')
plt.show()

In [17]:
#Set Up the Genetic Algorithm
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("hidden_units", np.random.randint, low=2, high=20, size=3)  #adjust the range and size
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.hidden_units)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=2, high=20, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [18]:
#Create the Initial Population
population = toolbox.population(n=10)  #adjust the population size

#Genetic Algorithm Loop
ngen = 10  # Number of generations
for gen in range(ngen):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.3)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

TypeError: ignored

In [19]:
#Extract Best Hyperparameters
best_individual = tools.selBest(population, k=1)[0]
best_hidden_units = best_individual[0]

In [20]:
#Train the MLP with Best Hyperparameters
best_mlp_model = build_mlp(best_hidden_units)
best_mlp_model.compile(optimizer='adam', loss='mean_squared_error')
best_mlp_model.fit(X_train, y_train, epochs=100, verbose=1)

TypeError: ignored

In [21]:
#Evaluate on Test Set
y_pred = best_mlp_model.predict(X_test)
mse = np.mean(np.square(y_pred - y_test))
print("Mean Squared Error:", mse)

NameError: ignored

In [ ]:
#comparison before and after GA

In [ ]:
# acc score